In [ ]:
import numpy as np
import torch
import torchvision
from torchvision.datasets import MNIST

In [ ]:
dataset = MNIST(root='data/', download=True)

In [ ]:
len(dataset)

In [ ]:
test_dataset = MNIST(root='data/', train=False)
print(len(test_dataset))

dataset[0]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
image, label = dataset[0]
plt.imshow(image)
print('Label: ', label.item())

In [ ]:
image, label = dataset[10]
plt.imshow(image)
print('Label: ', label.item())

In [ ]:
import torchvision.transforms as transforms

In [ ]:
dataset = MNIST(root='data/', train=True, transform=transforms.ToTensor())

In [ ]:
img_tensor, label = dataset[0]
print(img_tensor.shape)
print(label)

In [ ]:
print(img_tensor[:, 10:15, 10:15])
print(torch.max(img_tensor))
print(torch.min(img_tensor))

In [ ]:
plt.imshow(img_tensor[0], cmap='gray')

In [ ]:
def split_indices(n, val_pct):
  n_val = int(n * val_pct)
  idxs = np.random.permutation(n)
  return idxs[n_val:], idxs[:n_val]

In [ ]:
train_indices, val_indices = split_indices(len(dataset), val_pct=0.2)

In [ ]:
print(len(train_indices), len(val_indices))
print('sample val indices: ', val_indices[:20])

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader

In [ ]:
batch_size=100

train_sampler = SubsetRandomSampler(train_indices)
train_loader = DataLoader(dataset, batch_size, sampler=train_sampler)

val_sampler = SubsetRandomSampler(val_indices)
val_loader = DataLoader(dataset, batch_size, sampler=val_sampler)

In [ ]:
import torch.nn as nn

input_size = 28*28
num_classes = 10

model = nn.Linear(input_size, num_classes)

In [ ]:
print(model.weight.shape)
model.weight

In [ ]:
print(model.bias.shape)
model.bias

In [ ]:
for images, labels in train_loader:
  print(images.shape)
  outputs = model(images)
  break

In [ ]:
class MnistModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(input_size, num_classes)
    
  def forward(self, xb):
    xb = xb.reshape(-1, 28*28)
    out = self.linear(xb)
    return out
  
model = MnistModel()

In [ ]:
print(model.linear.weight.shape, model.linear.bias.shape)
list(model.parameters())

In [ ]:
for images, labels in train_loader:
  outputs = model(images)
  break
  
print('outputs.shape: ', outputs.shape)
print('Sample outputs: \n', outputs[:2].data)

In [ ]:
import torch.nn.functional as F

In [ ]:
probs = F.softmax(outputs, dim=1)

print("Sample probabilities: \n", probs[:2].data)
print("Sum: ", torch.sum(probs[0]).item())

In [ ]:
max_probs, preds = torch.max(probs, dim=1)
print(preds)

In [ ]:
labels

In [ ]:
def accuracy(l1, l2):
  return torch.sum(l1 == l2).item()/len(l1)

In [ ]:
accuracy(preds, labels)

In [ ]:
loss_fn = F.cross_entropy

In [ ]:
loss = loss_fn(outputs, labels)
print(loss)

In [ ]:
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [ ]:
def loss_batch(model, loss_func, xb, yb, opt = None, metric = None):
  preds = model(xb)
  loss = loss_func(preds, yb)
  
  if opt is not None:
    loss.backward()
    opt.step()
    opt.zero_grad()
    
  metric_result = None
  if metric is not None:
    metric_result = metric(preds, yb)
    
  return loss.item(), len(xb), metric_result

In [ ]:
def evaluate(model, loss_fn, valid_dl, metric=None):
  with torch.no_grad():
    results = [loss_batch(model, loss_fn, xb, yb, metric = metric) for xb,yb in valid_dl]
    
  losses, nums, metrics = zip(*results)
  total = np.sum(nums)
  total_loss = np.sum(np.multiply(losses, nums))
  avg_loss = total_loss / total
  avg_metric = None
  if metric is not None:
    tot_metric = np.sum(np.multiply(metrics, nums))
    avg_metric = tot_metric / total
  return avg_loss, total, avg_metric

In [ ]:
def accuracy(outputs, labels):
  _, preds = torch.max(outputs, dim=1)
  return torch.sum(preds == labels).item() / len(preds)

In [ ]:
val_loss, total, val_acc = evaluate(model, loss_fn, val_loader, metric = accuracy)
print('Loss: {:.4f}, Accuracy: {:.4f}'.format(val_loss, val_acc))

In [ ]:
def fit(epochs, model, loss_fn, opt, train_dl, valid_dl, metric=None):
  for epoch in range(epochs):
    for xb, yb in train_dl:
      loss,_,_ = loss_batch(model, loss_fn, xb, yb, opt)
      
    result = evaluate(model, loss_fn, valid_dl, metric)
    val_loss, total, val_metric = result
    
    if metric is None:
      print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, val_loss, metric.__name__, val_metric))
    else:
      print('Epoch [{}/{}], Loss: {:.4f}, {}: {:.4f}'.format(epoch+1, epochs, val_loss, metric.__name__, val_metric))

In [ ]:
model = MnistModel()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [ ]:
fit(5, model, F.cross_entropy, optimizer, train_loader, val_loader, accuracy)

In [ ]:
fit(5, model, F.cross_entropy, optimizer, train_loader, val_loader, accuracy)

In [ ]:
fit(5, model, F.cross_entropy, optimizer, train_loader, val_loader, accuracy)

In [ ]:
fit(5, model, F.cross_entropy, optimizer, train_loader, val_loader, accuracy)

In [ ]:
fit(5, model, F.cross_entropy, optimizer, train_loader, val_loader, accuracy)

In [ ]:
fit(5, model, F.cross_entropy, optimizer, train_loader, val_loader, accuracy)

In [ ]:
fit(10, model, F.cross_entropy, optimizer, train_loader, val_loader, accuracy)

In [ ]:
accuracies = [0.0790, 0.5989, 0.7224, 0.7653, 0.7851, 0.7994, 0.8093, 0.8162, 0.8225, 0.8266,
              0.8298, 0.8327, 0.8352, 0.8393, 0.8413, 0.8428, 0.8463, 0.8482, 0.8499, 0.8517,
              0.8532, 0.8546, 0.8560, 0.8573, 0.8582, 0.8592, 0.8598, 0.8608, 0.8619, 0.8628,
              0.8628, 0.8634, 0.8636, 0.8636, 0.8642, 0.8653, 0.8662, 0.8666, 0.8672, 0.8682, 0.8689]

plt.plot(accuracies, '-x')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('Accuracy vs. No. of eo=pochs')
plt.show()

In [ ]:
test_dataset = MNIST(root = 'data/', train = False, transform = transforms.ToTensor())

In [ ]:
img, label = test_dataset[0]
plt.imshow(img[0], cmap='gray')
print('Shape: ', img.shape)
print('Label: ', label.item())

In [ ]:
def predict_image(img, model):
  xb = img.unsqueeze(0)
  yb = model(xb)
  _, preds = torch.max(yb, dim=1)
  return preds[0].item()

In [ ]:
img, label = test_dataset[0]
plt.imshow(img[0], cmap = 'gray')
print('Label: ', label.item(), ', Predicted: ', predict_image(img, model))

In [ ]:
img, label = test_dataset[10]
plt.imshow(img[0], cmap = 'gray')
print('Label: ', label.item(), ', Predicted: ', predict_image(img, model))

In [ ]:
img, label = test_dataset[20]
plt.imshow(img[0], cmap = 'gray')
print('Label: ', label.item(), ', Predicted: ', predict_image(img, model))

In [ ]:
img, label = test_dataset[1889]
plt.imshow(img[0], cmap = 'gray')
print('Label: ', label.item(), ', Predicted: ', predict_image(img, model))

In [ ]:
test_loader = DataLoader(test_dataset, batch_size = 200)

test_loss, totla, test_acc = evaluate(model, loss_fn, test_loader, metric=accuracy)
print('Loss: {:.4f}, Accuracy: {:.4f}'.format(test_loss, test_acc))

In [ ]:
torch.save(model.state_dict(), 'mnist-logistic.pth')

In [ ]:
model.state_dict()

In [ ]:
model2 = MnistModel()
model2.load_state_dict(torch.load('mnist-logistic.pth'))
model2.state_dict()

In [ ]:
test_loss, total, test_acc = evaluate(model2, loss_fn, test_loader, metric = accuracy)
print('Loss: {:.4f}, Accuracy: {:.4f}'.format(test_loss, test_acc))